<a href="https://colab.research.google.com/github/asadnhasan/GenerativeAI-Projects/blob/main/End_to_End_PDF_Chat_using_LlamaIndex_%26_Llama_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installing the dependencies (Cassandra, LlamaIndex, PyPDF, tiktoken, and GradientAI)**

In [ ]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install -q llama-index
!pip install -q pypdf
!pip install -q tiktoken==0.4.0

## **Importing OS & JSON Modules for Accesing Gradient Workspace**

In [ ]:
import os
import json
from google.colab import userdata

os.environ['GRADIENT_ACCESS_TOKEN'] = userdata.get('GRADIENT_ACCESS_TOKEN')
os.environ['GRADIENT_WORKSPACE_ID'] = userdata.get('GRADIENT_WORKSPACE_ID')

## Importing Cassandra Driver and LlamaIndex

In [ ]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index import ServiceContext
from llama_index import set_global_service_context
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings import GradientEmbedding
from llama_index.llms import GradientBaseModelLLM
from llama_index.vector_stores import CassandraVectorStore

In [ ]:
import cassandra
print (cassandra.__version__)

3.29.0


In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# This secure connect bundle is autogenerated when you download your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-end-to-end-pdf-chat-using-llamaindex-llama-2.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("End to End PDF Chat using LlamaIndex & Llama 2-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(133690363125264) 97a54cfe-b400-4b48-9de0-b04f44c7b10b-us-east1.db.astra.datastax.com:29042:4139b007-aa45-41a0-b4b7-d39107002efc> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


4.0.11-c7d4aa9d7ae5


## Define the Gradient's Model Adapter for LLAMA-2

In [ ]:
llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)

## Retrieval Augmented Generation (RAG) with Gradient Embeddings (Configuration)

Embedding model: bge-large

In [ ]:
embed_model = GradientEmbedding(
    gradient_access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

service_context = ServiceContext.from_defaults(
    chunk_size=256, llm=llm, embed_model=embed_model
)

set_global_service_context(service_context)

## Loading the PDF into the Environment

In [ ]:
documents = SimpleDirectoryReader("/content/Documents").load_data()
print(f"Loaded {len(documents)} document(s).")

Loaded 116 document(s).


## Setting up Query and Index

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()

## Generate an answer to the query

In [ ]:
response = query_engine.query("Give me the details for Providing and fixing stainless steel(Grade304)railing made of Hollowtubes,channels,plates. Also give the rates")
print(response)


The details for Providing and fixing stainless steel (Grade 304) railing made of Hollow tubes, channels, plates are as follows:

17 10.28 DSR
Providing and fixing stainless steel (Grade 304) railing made of Hollow tubes, channels, plates etc., including welding, grinding, buffing, polishing and making curvature (wherever required) and fitting the same with necessary stainless steel nuts and bolts complete, i/c fixing the railing with necessary accessories & stainless steel dash fasteners, stainless steel bolts etc., of required size, on the top of the floor or the side of waist slab with suitable arrangement as per approval of Engineer-in-charge, (for payment purpose only weight of stainless steel members shall be considered excluding fixing accessories such as nuts, bolts, fasteners etc.).

Rates:
8489.25 K.G.
772.40
567.71
610.16
5179799.99

For walls with height more than 10m from ground level, the rates are as follows:

* For height of 10m to 13m, the rate is 324.75 sqm x 87.10 = 